In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_Tiergarten_multi_class

In [2]:
numNodes = Compute_Jacobian_Tiergarten_multi_class.numNodes;
numLinks = Compute_Jacobian_Tiergarten_multi_class.numLinks;
numODpairs = Compute_Jacobian_Tiergarten_multi_class.numODpairs;
numZones = Compute_Jacobian_Tiergarten_multi_class.numZones;
od_pairs = Compute_Jacobian_Tiergarten_multi_class.od_pairs;
link_list_js = Compute_Jacobian_Tiergarten_multi_class.link_list_js;
link_length_list = Compute_Jacobian_Tiergarten_multi_class.link_length_list;

In [3]:
include("../../Julia_files/initia_data.jl");
include("prepare_data_Tiergarten.jl");
include("extract_data_Tiergarten.jl");
include("../../Julia_files/tap_MSA_multi_class.jl");
include("../../Julia_files/inverseVI_multi_class.jl");
include("../../Julia_files/demands_adjustment_gradi_multi_class.jl");

In [4]:
demandsDictCar[0] = demandsDict[0]["car"];
demandsDictCar[1] = demandsDict[1]["car"];
demandsDictTruck[0] = demandsDict[0]["truck"];
demandsDictTruck[1] = demandsDict[1]["truck"];

In [5]:
demandsDiffDictCar[1] = norm(demandsDicToVec(demandsDictCar[1]) - demandsDicToVec(demandsDictCar[0]))/
norm(demandsDicToVec(demandsDictCar[0]));

demandsDiffDictTruck[1] = norm(demandsDicToVec(demandsDictTruck[1]) - demandsDicToVec(demandsDictTruck[0]))/
norm(demandsDicToVec(demandsDictTruck[0]));

In [6]:
demandsVecDictCar[0] = demandsDicToVec(demandsDictCar[0]);
demandsVecDictTruck[0] = demandsDicToVec(demandsDictTruck[0]);

In [7]:
demandsVecDictCar

Dict{Any,Any} with 1 entry:
  0 => [18.072,14.656,17.456,12.176,2.984,0.08,0.248,0.976,0.416,0.168  …  5.99…

In [8]:
demandsVecDictTruck

Dict{Any,Any} with 1 entry:
  0 => [4.518,3.664,4.364,3.044,0.746,0.02,0.062,0.244,0.104,0.042  …  1.498,2.…

In [9]:
fcoeffs = [1, 0, 0, 0, 1];

# get observed flow vector (corresponding to ground truth demands and ground truth costs)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA_Multi(demandsDictCar[0], demandsDictTruck[0], fcoeffs);

In [10]:
tapFlowDicDict[0]

Dict{Any,Any} with 2 entries:
  "car"   => Dict{Any,Any}(Pair{Any,Any}((248,65),395.656),Pair{Any,Any}((137,1…
  "truck" => Dict{Any,Any}(Pair{Any,Any}((248,65),98.914),Pair{Any,Any}((137,10…

In [11]:
tapFlowVecDict[0]

2×766 Array{Float64,2}:
 32.9041  118.439   612.298  0.0  547.093   …  3.17326   29.9437   58.9791
  8.224    29.6097  152.431  0.0   97.4636     0.793226   7.48221  14.7385

In [12]:
# get arcs data corresponding to ground truth demands and flows
arcsDict[0] = observFlow("./data_Tiergarten/Tiergarten_net.txt", tapFlowDicDict[0]);

In [13]:
arcsDict[0]

Dict{Any,Any} with 766 entries:
  (248,65)  => Arc(248,65,900.0,17.0,593.484,395.656,98.914)
  (137,106) => Arc(137,106,2800.0,1.66667,268.764,179.176,44.794)
  (267,256) => Arc(267,256,2400.0,7.33333,104.42,69.6041,17.4079)
  (354,353) => Arc(354,353,900.0,1.0,344.967,229.883,57.5423)
  (56,55)   => Arc(56,55,600.0,0.333333,0.0,0.0,0.0)
  (259,255) => Arc(259,255,2400.0,0.666667,0.0,0.0,0.0)
  (125,177) => Arc(125,177,900.0,1.66667,0.0,0.0,0.0)
  (36,35)   => Arc(36,35,2800.0,0.333333,23.5704,15.7155,3.92747)
  (183,184) => Arc(183,184,900.0,0.333333,0.0,0.0,0.0)
  (43,361)  => Arc(43,361,2800.0,12.6667,7.81583,5.21975,1.29804)
  (16,277)  => Arc(16,277,999999.0,0.0,1833.91,1221.37,306.267)
  (298,307) => Arc(298,307,2400.0,1.0,1799.71,1200.47,299.619)
  (31,34)   => Arc(31,34,2800.0,0.666667,130.308,86.8762,21.7158)
  (187,191) => Arc(187,191,600.0,0.666667,274.182,182.788,45.6969)
  (241,244) => Arc(241,244,2400.0,0.333333,0.0,0.0,0.0)
  (84,95)   => Arc(84,95,900.0,2.33333,0.0,0.0,

In [14]:
deg_list = [3, 4, 5, 6]
c_list = [0.5, 1.0, 1.5]
lam_list = [0.01, 0.1, 1.0, 10.0, 100.0]

for deg in deg_list
    for c in c_list
        for lam in lam_list
            coeffs_dict[(deg, c, lam, 1)], objInvVIDict[1] = train(lam, deg, c, demandsDictCar[0], 
            demandsDictTruck[0], arcsDict[0]);
        end
    end
end

In [15]:
outfile = open("./results_Tiergarten/coeffs_dict_Tiergarten.json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)